In [1]:
import arviz as az
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [2]:
# Read in data and display first 5 lines
data = pd.read_csv("/mnt/d/PPRV_lineage_IV/data/stratified_proximity_data.csv")

N = [6, 6, 6 , 6, 6, 5, 6,5, 6, 6, 6]

data.head()

,experiment,duration,seeder,Cap2,time_between_0.05_and_0.5m,time_between_0.5_and_1m,time_between_1_and_2m,time_above_2m,infected
0,1,1,436,100,2.783333,10.600000,23.483333,25.850000,0
1,1,1,436,116,4.500000,20.983333,23.216667,13.916667,0
2,1,1,436,255,6.200000,20.900000,27.200000,8.416667,0
3,1,1,436,449,17.016667,20.733333,15.933333,9.050000,0
4,1,1,436,3008,0.183333,1.033333,9.700000,51.816667,0


In [5]:
traces = []
for i in range(1, 6):
    filename = f"/mnt/d/PPRV_LINEAGE_IV/output/InferenceData/trace_model{i}.pkl"
    with open(filename, "rb") as file:
        trace = pickle.load(file)
        traces.append(trace)


trace_model1 = traces[0]
trace_model2 = traces[1]
trace_model3 = traces[2]
trace_model4 = traces[3]     
trace_model5 = traces[4]     


In [6]:
def process_simulations(trace, df, N):
    """
    Process Bernoulli simulations from a PyMC trace and original dataframe.
    
    Parameters:
    trace (arviz.InferenceData): The trace from a PyMC model.
    df (pd.DataFrame): Original dataframe with 'experiment', 'duration', and 'infected' columns.
    N (list): List of sample sizes for each experiment.
    
    Returns:
    pd.DataFrame: Processed dataframe with cases per experiment.
    """
    try:
        # Extract posterior samples
        samples = trace.posterior.stack(sample=("chain", "draw"))
        
        # Create probability dataframe
        p_list_df = pd.DataFrame(samples['p_i'].values)
        p_list_df.columns = p_list_df.add_prefix('pred_').columns
        
        # Perform Bernoulli simulations
        simulations = p_list_df.apply(lambda prob: np.random.binomial(1, prob))
        
        # Combine original data with simulations
        result_df = pd.concat([
            df[["experiment", "duration", "infected"]].reset_index(drop=True),
            simulations.reset_index(drop=True)
        ], axis=1)
        
        # Calculate cases per experiment
        cases_per_exp = result_df.groupby(['experiment', 'duration']).sum().reset_index()
        
        # Add sample sizes and rename columns
        cases_per_exp.insert(2, 'N', N)
        cases_per_exp = cases_per_exp.rename(columns={'infected': 'obs'})
        
        return cases_per_exp
    
    except KeyError as e:
        print(f"Error: Missing expected column in dataframe or trace. {str(e)}")
    except ValueError as e:
        print(f"Error: Mismatch in data shapes or types. {str(e)}")
    except Exception as e:
        print(f"An unexpected error occurred: {str(e)}")

In [7]:
# processed_data_1 = process_simulations(trace_model1, data, N)
# processed_data_2 = process_simulations(trace_model2, data, N)
# processed_data_3 = process_simulations(trace_model3, data, N)
# processed_data_4 = process_simulations(trace_model4, data, N)
# processed_data_5 = process_simulations(trace_model5, data, N)

In [8]:
processed_data_5

,experiment,duration,N,obs,pred_0,pred_1,pred_2,pred_3,pred_4,pred_5,...,pred_99990,pred_99991,pred_99992,pred_99993,pred_99994,pred_99995,pred_99996,pred_99997,pred_99998,pred_99999
0,1,1,6,0,1,0,1,0,0,0,...,0,0,1,0,1,1,0,2,0,0
1,1,6,6,0,2,0,1,0,2,1,...,1,2,0,1,1,3,0,0,3,0
2,1,24,6,0,2,3,4,0,2,1,...,2,3,1,0,1,3,0,3,2,5
3,2,24,6,1,1,0,1,0,1,1,...,0,0,1,1,0,1,0,0,0,0
4,2,44,6,4,5,3,2,3,3,2,...,3,3,3,3,3,2,3,3,4,4
5,3,24,5,3,2,1,0,1,0,1,...,1,0,2,0,0,4,0,1,3,2
6,4,24,6,0,2,2,0,0,1,0,...,0,0,1,0,0,1,1,0,0,1
7,4,48,5,4,4,0,2,0,4,0,...,2,2,1,1,3,4,1,2,3,3
8,5,24,6,1,6,3,2,1,3,5,...,4,2,1,4,3,2,1,3,0,4
9,5,48,6,3,2,1,2,0,4,2,...,4,1,2,3,4,4,1,2,2,3


In [9]:
# processed_data_1.to_csv('/mnt/d/PPRV_lineage_IV/output/model_predictions/model1_pred.csv', index=False)
# processed_data_2.to_csv('/mnt/d/PPRV_lineage_IV/output/model_predictions/model2_pred.csv', index=False)
# processed_data_3.to_csv('/mnt/d/PPRV_lineage_IV/output/model_predictions/model3_pred.csv', index=False)
# processed_data_4.to_csv('/mnt/d/PPRV_lineage_IV/output/model_predictions/model4_pred.csv', index=False)
# processed_data_5.to_csv('/mnt/d/PPRV_lineage_IV/output/model_predictions/model5_pred.csv', index=False)